In [14]:
from datetime import datetime
datetime.fromtimestamp(1626391901000/ 1e3).strftime("%H:%M:%S")

'00:31:41'

In [4]:
import pandas as pd
import re
import os

In [9]:
stop_times = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "stop_times.txt", delimiter=",")
stops = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs" + os.sep +  "stops.txt", delimiter=",")
routes = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "routes.txt", delimiter=",")
trips = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "trips.txt", delimiter=",")
calendar = pd.read_csv("."+ os.sep + "RAW_DATA"+ os.sep + "gtfs"+ os.sep + "calendar.txt", delimiter=",")

In [10]:
stop_times["arrival_time"] = pd.to_timedelta(stop_times["arrival_time"])

In [15]:
departure = "00:31:41"
origin = "UCD N11 Entrance"
destination = "Dawson Street, stop 793"
day = "thursday"
bus_route = "11"

In [16]:
days = list(calendar[calendar[day] == 1]["service_id"].values)

In [17]:
day_trips = set(trips[trips["service_id"].isin(days)]["trip_id"].values)

In [18]:
route_trips = set(trips[trips["route_id"].isin(list(routes[routes["route_short_name"] == bus_route]["route_id"].values))]["trip_id"].values)

In [21]:
start_trips = stops[stops["stop_name"] == origin]["stop_id"].values[0]
end_trips = stops[stops["stop_name"] == destination]["stop_id"].values[0]

In [ ]:
start_trips  = set(stop_times[stop_times["stop_id"] == start]["trip_id"].values)
end_trips  = set(stop_times[stop_times["stop_id"] == end]["trip_id"].values)

In [ ]:
r = start_trips.intersection(end_trips)
r = r.intersection(route_trips)
r = r.intersection(day_trips)

In [ ]:
I = None
O = None

for trip in r:
    if re.search(".I$", trip):
        I = trip
        break
        
for trip in r:
    if re.search(".O$", trip):
        O = trip
        break
print(O)
print(I)

In [ ]:
if O != None and ((stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == start)]["stop_sequence"].values[0]) < (stop_times[(stop_times["trip_id"] == O) & (stop_times["stop_id"] == end)]["stop_sequence"].values[0])):
    correct_direction = [trip for trip in r if re.search(".O$", trip)]
elif I != None and ((stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == start)]["stop_sequence"].values[0]) < (stop_times[(stop_times["trip_id"] == I) & (stop_times["stop_id"] == end)]["stop_sequence"].values[0])):
    correct_direction = [trip for trip in r if re.search(".I$", trip)]

In [ ]:
final = stop_times[(stop_times["trip_id"].isin(correct_direction)) & (stop_times["stop_id"] == start)][["trip_id","arrival_time"]]
final["arrival_time"] = abs(final["arrival_time"] - pd.to_timedelta(departure))
final = final.sort_values(by="arrival_time")

In [ ]:
final

In [ ]:
tripid = final.iloc[0]["trip_id"]

In [ ]:
seq = stop_times[stop_times["trip_id"] == tripid][["stop_id", "stop_sequence"]].sort_values(by="stop_sequence")

In [ ]:
rstart = seq[seq["stop_id"] == start].index[0]

In [ ]:
rfin = seq[seq["stop_id"] == end].index[0]

In [ ]:
test = seq.loc[rstart:rfin]

In [ ]:
final = test.merge(stops, how='left', on='stop_id')

In [ ]:
flist = list(final["stop_name"].values)
flist = [stop.split(" stop ")[1] for stop in flist]
flist